First updated @ Oct 31, 2021      Happy Halloween👻

Last updated @ Nov 2, 2021

In [2]:
# import library
import pandas as pd

## Read all movies

- get the full movie list and the imdbid

In [57]:
df = pd.read_json('http://bechdeltest.com/api/v1/getAllMovies')

In [58]:
df.tail()

,year,imdbid,title,rating,id
9324,2021,9421570,"Guilty, The",1,10079
9325,2021,11804152,Till Death,0,10083
9326,2021,4244994,"Last Duel, The",3,10085
9327,2021,7097896,Venom Let there be Carnage,2,10092
9328,2021,2452150,Respect,3,10106


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9329 entries, 0 to 9328
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   imdbid  9329 non-null   object
 1   rating  9329 non-null   int64 
 2   id      9329 non-null   int64 
 3   year    9329 non-null   int64 
 4   title   9329 non-null   object
dtypes: int64(3), object(2)
memory usage: 364.5+ KB


In [44]:
# save dataframe to a csv file
df.to_csv('Bechdel.csv')

## Get movie by imdbid

- then loop through the imdbid in the previous dataframe to get each movie info
- create a new dataframe and output the newer csv file

Note: The website is fragile and you may experience timeout error. Make sure to save dataframe to csv so that you don't lose everything.

In [59]:
# create an empty dataframe
df_new = pd.DataFrame(columns=['title', 'year', 'rating', 'dubious', 'imdbid', 'id', 'submitterid', 'date', 'visible'])
df_new

,title,year,rating,dubious,imdbid,id,submitterid,date,visible


In [71]:
count = 1

# call from the latest to the earliest movie, in case the website crashes
for imdbid in df['imdbid'][-1::-1]:
    series = pd.read_json('http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=' + str(imdbid), typ='series')
    # print(series)
    df_new = df_new.append(series, ignore_index=True)
    # every 10 movies, we save to a csv file, 
    # so that we wouldn't lose everything when there is an error
    if count % 10 == 0:
        df_new.to_csv('Bechdel_detailed_reversed.csv')
    count += 1
df_new.to_csv('Bechdel_detailed_reversed.csv')

This took about 7 hours to execute

In [ ]:
# reverse 
df_new = df_new.iloc[::-1].reset_index(drop=True)
df_new.to_csv('Bechdel_detailed.csv', index=False)

## Check data integrity

- check duplicates
- check row numbers
- check index

In [12]:
df = pd.read_csv('./Bechdel.csv', index_col=0)
df_to_check = pd.read_csv('./Bechdel_detailed.csv')
df_to_check

,title,year,rating,dubious,imdbid,id,submitterid,date,visible
0,Passage de Venus,1874.0,0.0,0.0,3155794.0,9602.0,18880.0,2021-04-02 20:58:09,1.0
1,La Rosace Magique,1877.0,0.0,0.0,14495706.0,9804.0,19145.0,2021-05-11 00:11:22,1.0
2,Sallie Gardner at a Gallop,1878.0,0.0,0.0,2221420.0,9603.0,18882.0,2021-04-03 02:25:27,1.0
3,Le singe musicien,1878.0,0.0,0.0,12592084.0,9806.0,19151.0,2021-05-11 23:38:54,1.0
4,Athlete Swinging a Pick,1881.0,0.0,0.0,7816420.0,9816.0,19162.0,2021-05-13 01:32:14,1.0
...,...,...,...,...,...,...,...,...,...
9324,"Guilty, The",2021.0,1.0,0.0,9421570.0,10079.0,19599.0,2021-10-10 20:16:13,1.0
9325,Till Death,2021.0,0.0,0.0,11804152.0,10083.0,19613.0,2021-10-18 19:58:48,1.0
9326,"Last Duel, The",2021.0,3.0,1.0,4244994.0,10085.0,19619.0,2021-10-20 02:30:29,1.0
9327,Venom Let there be Carnage,2021.0,2.0,0.0,7097896.0,10092.0,19627.0,2021-10-21 10:11:03,1.0


In [13]:
def check_dataset(df_to_check):
    lengthFlag = len(df_to_check) == len(df)
    print(f'Same number of rows? {lengthFlag}, should be {len(df)} movies, got {len(df_to_check)} movies')
    if lengthFlag:
        movieFlag = True
        for i in range(len(df)):
            if df['imdbid'].iloc[i] != df_to_check['imdbid'].iloc[i]:
                print(i, df['imdbid'].iloc[i], df_to_check['imdbid'].iloc[i])
                movieFlag = False
                break
        print(f'Same movies? {movieFlag}')
    else:
        print(f'Same movies? False')

In [14]:
# check integrity and fail
check_dataset(df_to_check)

Same number of rows? True, should be 9329 movies, got 9329 movies
Same movies? True
